In [34]:
import pandas as pd

file_path = r"c_elegans.PRJNA13758.WS293.geneOtherIDs.txt\c_elegans.PRJNA13758.WS293.geneOtherIDs.txt"
df = pd.read_csv(file_path, sep='\t', header=None, names=['Wormbase_ID', 'Status', 'Sequence_Name', 'Gene_Name', 'Other_Name'])
#df_filtered= df[df['Status'] != 'Dead']



,Wormbase_ID,Status,Sequence_Name,Gene_Name,Other_Name
0,WBGene00000001,Live,Y110A7A.10,aap-1,CELE_Y110A7A.10
1,WBGene00000002,Live,F27C8.1,aat-1,CELE_F27C8.1
2,WBGene00000003,Live,F07C3.7,aat-2,CELE_F07C3.7
3,WBGene00000004,Live,F52H2.2,aat-3,CELE_F52H2.2
4,WBGene00000005,Live,T13A10.10,aat-4,CELE_T13A10.10
...,...,...,...,...,...
52104,WBGene00306126,Live,Y54F10AM.16,cone-1,CELE_Y54F10AM.16
52105,WBGene00306131,Live,Y34B4A.20,NaN,CELE_Y34B4A.20
52106,WBGene00306132,Live,F54D10.10,NaN,CELE_F54D10.10
52107,WBGene00306133,Live,F42G4.11,azyx-1,CELE_F42G4.11


In [103]:
# 取出所有可能的名稱和對應的 Gene_ID


# Combine Sequence_Name, Gene_Name, and Other_Name into one DataFrame for easier processing
all_names = pd.concat([
    df[['Wormbase_ID', 'Sequence_Name']].rename(columns={'Sequence_Name': 'Name'}),
    df[['Wormbase_ID', 'Gene_Name']].rename(columns={'Gene_Name': 'Name'}),
    df[['Wormbase_ID', 'Other_Name']].rename(columns={'Other_Name': 'Name'})
])


# Remove rows with empty or NaN names
all_names.dropna(subset=['Name'], inplace=True)
duplicated_names = all_names[all_names.duplicated(subset=['Name'], keep=False)]
print(duplicated_names)
# Group by Name and Wormbase_ID, and count occurrences
name_grouped = all_names.groupby(['Name', 'Wormbase_ID']).size().reset_index(name='Count')

# Find names that appear in more than one Wormbase_ID
duplicates = name_grouped.groupby('Name')['Wormbase_ID'].nunique().reset_index()
duplicates_list = duplicates[duplicates['Wormbase_ID'] > 1]['Name'].unique()

# Filter the original df for rows where the names are in the duplicates list
duplicate_rows = df[
    df['Sequence_Name'].isin(duplicates_list) |
    df['Gene_Name'].isin(duplicates_list) |
    df['Other_Name'].isin(duplicates_list)
]



                

          Wormbase_ID            Name
420    WBGene00000481         ZC416.8
1081   WBGene00001147       Y105E8A.7
1542   WBGene00001608         R07B1.8
1976   WBGene00002042        H26D21.1
2911   WBGene00002977       Y105E8A.7
...               ...             ...
50623  WBGene00235206              U2
50651  WBGene00235234    Y57G11C.1147
50665  WBGene00235248        ZK105.14
50743  WBGene00235327       T02B11.10
50930  WBGene00269374  CELE_F35F10.18

[443 rows x 2 columns]


array([], shape=(0, 152), dtype=object)

In [96]:

from collections import defaultdict


# 初始化字典來儲存重複名稱與其對應的 Wormbase_ID
name_to_ids = defaultdict(list)

# 遍歷 duplicate_rows，將重複的名稱與對應的 Wormbase_ID 儲存
for idx, row in duplicate_rows.iterrows():
    for name in ['Gene_Name', 'Sequence_Name', 'Other_Name']:
        if row[name] in duplicates_list:
            name_to_ids[row[name]].append(row['Wormbase_ID'])

# 構建結果，將重複的名稱、出現次數及對應的 Wormbase_ID 組合成以逗號分隔的字串
result = []
for name, ids in name_to_ids.items():
    ids_str = ",".join(ids)  # 將 Wormbase_ID 列表合併成逗號分隔的字串
    row = [name, len(ids), ids_str]  # 名稱, 出現次數, 對應的 Wormbase_ID 列表
    result.append(row)

# 構建 DataFrame，將所有 Wormbase_ID 放在一個欄位中
result_df = pd.DataFrame(result, columns=['Name', 'Count', 'Wormbase_IDs'])
result_df.to_csv('duplicates_v2.csv', index=False)

# 顯示結果供確認
result_df.head(10)


,Name,Count,Wormbase_IDs
0,kap-1,2,"WBGene00000101,WBGene00002182"
1,ref-1,2,"WBGene00000120,WBGene00004334"
2,ref-2,2,"WBGene00000121,WBGene00004335"
3,arl-5,2,"WBGene00000189,WBGene00001358"
4,arl-6,2,"WBGene00000190,WBGene00000193"
5,atg-2,2,"WBGene00000225,WBGene00019748"
6,K04C2.a,2,"WBGene00000265,WBGene00019380"
7,cah-1,2,"WBGene00000279,WBGene00000294"
8,cep-1,3,"WBGene00000366,WBGene00000467,WBGene00004048"
9,cone-1,2,"WBGene00000464,WBGene00306126"


### part2




## 2-1

In [19]:
import gzip
from collections import defaultdict

file_path = 'c_elegans.PRJNA13758.WS293.mRNA_transcripts.fa'
transcript_names = []
gene_ids = []

# Read the file and extract transcript names and gene IDs
with open(file_path, 'r') as file:
    for line in file:
        line = line.strip()
        if line.startswith('>'):  # Identify the header lines
            # Split the line to extract the transcript name and gene ID
            header = line.split()
            #print(header)
            transcript_name = header[0][1:]  # Remove '>' and take the first part as transcript name
            gene_id = header[1].split('=')[1]  # Extract the gene ID part after "gene="
            
            # Append to lists
            transcript_names.append(transcript_name)
            gene_ids.append(gene_id)

# Combine the extracted information into a DataFrame
extracted_data = pd.DataFrame({
    'Gene_ID': gene_ids,
    'Transcript_Name': transcript_names,
    
})
extracted_data['Type'] = 'coding_transcript'
extracted_data

,Gene_ID,Transcript_Name,Type
0,WBGene00007063,2L52.1a.1,coding_transcript
1,WBGene00007063,2L52.1b.1,coding_transcript
2,WBGene00007064,2RSSE.1a.1,coding_transcript
3,WBGene00007064,2RSSE.1b.1,coding_transcript
4,WBGene00007064,2RSSE.1c.1,coding_transcript
...,...,...,...
32012,WBGene00022842,ZK994.6b.1,coding_transcript
32013,WBGene00008352,cTel54X.1.1,coding_transcript
32014,WBGene00305997,cTel54X.2.1,coding_transcript
32015,WBGene00007068,cTel55X.1a.1,coding_transcript


## 2-2

In [20]:
# Initialize lists to store extracted data
transcript_names = []
gene_ids = []
biotypes = []

# Define the file path for the new file
file_path = 'c_elegans.PRJNA13758.WS293.ncRNA_transcripts.fa'

# Read the file and extract transcript names, gene IDs, and biotype
with open(file_path, 'r') as file:
    for line in file:
        line = line.strip()
        if line.startswith('>'):  # Identify header lines
            # Split the line to extract the transcript name, biotype, and gene ID
            header = line.split()
            transcript_name = header[0][1:]  # Remove '>' to get the transcript name
            biotype = header[1].split('=')[1]  # Extract biotype from "biotype="
            gene_id = header[2].split('=')[1]  # Extract gene ID from "gene="
            
            # Append to lists
            transcript_names.append(transcript_name)
            gene_ids.append(gene_id)
            biotypes.append(biotype)

# Combine the extracted information into a DataFrame
extracted_data_ncRNA = pd.DataFrame({
    'Gene_ID': gene_ids,
    'Transcript_Name': transcript_names,
    'Type': biotypes
})
extracted_data_ncRNA

,Gene_ID,Transcript_Name,Type
0,WBGene00200402,2L52.2,ncRNA
1,WBGene00195865,2RSSE.4,ncRNA
2,WBGene00197051,2RSSE.5,ncRNA
3,WBGene00199694,2RSSE.6,ncRNA
4,WBGene00199940,2RSSE.7,ncRNA
...,...,...,...
26689,WBGene00200245,ZK1290.21,ncRNA
26690,WBGene00201513,ZK1290.22,ncRNA
26691,WBGene00305939,ZK1307.10,circular_ncRNA
26692,WBGene00305940,ZK1307.11,circular_ncRNA


## 2-3

In [ ]:
# Initialize lists to store extracted data
transcript_names = []
gene_ids = []
biotypes = []

# Define the file path for the new file
file_path = 'c_elegans.PRJNA13758.WS293.pseudogenic_transcripts.fa'

# Read the file and extract transcript names, gene IDs, and biotype
with open(file_path, 'r') as file:
    for line in file:
        line = line.strip()
        if line.startswith('>'):  # Identify header lines
            # Split the line to extract the transcript name, biotype, and gene ID
            header = line.split()
            transcript_name = header[0][1:]  # Remove '>' to get the transcript name
            gene_id = header[1].split('=')[1]  # Extract gene ID from "gene="
            
            # Append to lists
            transcript_names.append(transcript_name)
            gene_ids.append(gene_id)
            biotypes.append("pseudogenic_transcripts")  # Add biotype as pseudogenic_transcripts

# Combine the extracted information into a DataFrame
extracted_data_pseudogenic = pd.DataFrame({
    'Gene_ID': gene_ids,
    'Transcript_Name': transcript_names,
    'Type': biotypes
})


## 2-4


In [48]:
# Initialize lists to store extracted data
transcript_names = []
gene_ids = []
biotypes = []

# Define the file path for the new file
file_path = 'c_elegans.PRJNA13758.WS293.transposon_transcripts.fa'

# Read the file and extract transcript names, gene IDs, and biotype
with open(file_path, 'r') as file:
    for line in file:
        line = line.strip()
        if line.startswith('>'):  # Identify header lines
            # Split the line to extract the transcript name, biotype, and gene ID
            header = line.split()
            transcript_name = header[0][1:]  # Remove '>' to get the transcript name
            biotype = header[1].split('=')[1]  # Extract biotype from "biotype="
            gene_id = header[2].split('=')[1]  # Extract gene ID from "gene="
            
            # Append to lists
            transcript_names.append(transcript_name)
            gene_ids.append(gene_id)
            biotypes.append(biotype)

# Combine the extracted information into a DataFrame
extracted_data_ncRNA_transcripts = pd.DataFrame({
    'Gene_ID': gene_ids,
    'Transcript_Name': transcript_names,
    'Type': biotypes
})
extracted_data,extracted_data_ncRNA,extracted_data_pseudogenic,extracted_data_ncRNA_transcripts

(              Gene_ID Transcript_Name               Type
 0      WBGene00007063       2L52.1a.1  coding_transcript
 1      WBGene00007063       2L52.1b.1  coding_transcript
 2      WBGene00007064      2RSSE.1a.1  coding_transcript
 3      WBGene00007064      2RSSE.1b.1  coding_transcript
 4      WBGene00007064      2RSSE.1c.1  coding_transcript
 ...               ...             ...                ...
 32012  WBGene00022842      ZK994.6b.1  coding_transcript
 32013  WBGene00008352     cTel54X.1.1  coding_transcript
 32014  WBGene00305997     cTel54X.2.1  coding_transcript
 32015  WBGene00007068    cTel55X.1a.1  coding_transcript
 32016  WBGene00007068    cTel55X.1b.1  coding_transcript
 
 [32017 rows x 3 columns],
               Gene_ID Transcript_Name            Type
 0      WBGene00200402          2L52.2           ncRNA
 1      WBGene00195865         2RSSE.4           ncRNA
 2      WBGene00197051         2RSSE.5           ncRNA
 3      WBGene00199694         2RSSE.6           ncRNA


In [81]:
# Check for duplicate Gene_IDs between extracted_data_ncRNA and extracted_data_transposon_mrna
duplicate_gene_ids = pd.merge(extracted_data_ncRNA[['Gene_ID']], extracted_data_ncRNA_transcripts[['Gene_ID']], on='Gene_ID', how='inner')
duplicate_gene_ids

,Gene_ID
0,WBGene00195014
1,WBGene00195015


In [75]:
try:
    # Combine the two DataFrames into one
    combined_data = pd.concat([extracted_data, extracted_data_ncRNA,extracted_data_ncRNA_transcripts,extracted_data_pseudogenic], ignore_index=True)
except NameError as e:
    # Handle the case where one or both DataFrames do not exist
    combined_data = pd.DataFrame()
    print(f"Error: {e}")
combined_data.to_csv("combined_data.csv",index=False)

In [50]:
# Check if combined_data is available and contains the 'Type' column
try:
    # Group by 'Type' and count the number of occurrences for each type
    type_counts = combined_data['Type'].value_counts().reset_index()
    type_counts.columns = ['Type', 'Number']
    
    # Calculate the sum of all counts
    total_count = type_counts['Number'].sum()
    
    # Add a row for the sum at the bottom of the table
    total_row = pd.DataFrame({'Type': ['SUM'], 'Number': [total_count]})
    summary_table = pd.concat([type_counts, total_row], ignore_index=True)
    
   
except Exception as e:
    print(f"Error: {e}")
summary_table.to_csv('RNA_type_summary_v2.csv',index=False)

### part3


In [23]:
# Ensure combined_data is available and populated
try:
    # Merge combined_data with df_filtered using Gene_ID and Wormbase_ID
    merged_data = pd.merge(combined_data, df, left_on='Gene_ID', right_on='Wormbase_ID', how='left')
    
    # Ensure the DataFrame has the required seven columns
    merged_data = merged_data[['Gene_ID',  'Status', 'Sequence_Name', 'Gene_Name', 'Other_Name','Transcript_Name', 'Type']]
    sorted_data = merged_data.sort_values(by='Gene_ID', ascending=True).reset_index(drop=True)

    
except Exception as e:
    print(f"Error: {e}")
sorted_data
sorted_data.to_csv('big_table_output.csv', index=False)

In [29]:
# 使用外連接進行合併，確保所有 Gene_ID 和 Wormbase_ID 都被保留
merged_data = pd.merge(combined_data, df, left_on='Gene_ID', right_on='Wormbase_ID', how='outer')

# 如果 Gene_ID 是空的，則用 Wormbase_ID 填充 Gene_ID
merged_data['Gene_ID'] = merged_data['Gene_ID'].fillna(merged_data['Wormbase_ID'])

# 刪除不再需要的 Wormbase_ID 欄位
merged_data.drop(columns=['Wormbase_ID'], inplace=True)

# 將缺失的資料填充為空字串
merged_data.fillna('', inplace=True)
merged_data = merged_data[['Gene_ID',  'Status', 'Sequence_Name', 'Gene_Name', 'Other_Name','Transcript_Name', 'Type']]
# 將結果儲存為 CSV 檔案

merged_data.to_csv('big_table_v2.csv', index=False)
merged_data

,Gene_ID,Status,Sequence_Name,Gene_Name,Other_Name,Transcript_Name,Type
0,WBGene00000001,Live,Y110A7A.10,aap-1,CELE_Y110A7A.10,Y110A7A.10.1,coding_transcript
1,WBGene00000002,Live,F27C8.1,aat-1,CELE_F27C8.1,F27C8.1.1,coding_transcript
2,WBGene00000003,Live,F07C3.7,aat-2,CELE_F07C3.7,F07C3.7.1,coding_transcript
3,WBGene00000004,Live,F52H2.2,aat-3,CELE_F52H2.2,F52H2.2a.1,coding_transcript
4,WBGene00000004,Live,F52H2.2,aat-3,CELE_F52H2.2,F52H2.2b.1,coding_transcript
...,...,...,...,...,...,...,...
62723,WBGene00306126,Live,Y54F10AM.16,cone-1,CELE_Y54F10AM.16,Y54F10AM.16b.1,coding_transcript
62724,WBGene00306131,Live,Y34B4A.20,,CELE_Y34B4A.20,Y34B4A.20.1,coding_transcript
62725,WBGene00306132,Live,F54D10.10,,CELE_F54D10.10,F54D10.10.1,coding_transcript
62726,WBGene00306133,Live,F42G4.11,azyx-1,CELE_F42G4.11,F42G4.11.1,coding_transcript


In [74]:
import pandas as pd

# 使用外連接進行合併，確保所有 Gene_ID 和 Wormbase_ID 都被保留
merged_data = pd.merge(combined_data, df, left_on='Gene_ID', right_on='Wormbase_ID', how='outer')

# 如果 Gene_ID 是空的，則用 Wormbase_ID 填充 Gene_ID
merged_data['Gene_ID'] = merged_data['Gene_ID'].fillna(merged_data['Wormbase_ID'])

# 刪除不再需要的 Wormbase_ID 和 Type 欄位
merged_data.drop(columns=['Wormbase_ID', 'Type'], inplace=True)

# 將缺失的資料填充為空字串
merged_data.fillna('', inplace=True)

# 按照 Gene_ID 進行分組，並合併多個 Transcript_Name，忽略 Type
grouped = merged_data.groupby(['Gene_ID', 'Status', 'Sequence_Name', 'Gene_Name', 'Other_Name'], as_index=False).agg({
    'Transcript_Name': lambda x: ','.join(x),  # 將所有 Transcript_Name 合併成一個字串
})

# 計算每個 Gene_ID 下有多少個 transcript，忽略空值
grouped['Transcript_Count'] = grouped['Transcript_Name'].apply(lambda x: 0 if x == '' else len(x.split(',')))


# 調整列順序：將 Transcript_Count 放到第六列，將 Transcript_Name 放到第七列
grouped = grouped[['Gene_ID', 'Status', 'Sequence_Name', 'Gene_Name', 'Other_Name', 'Transcript_Count', 'Transcript_Name']]

# 將結果儲存為 CSV 檔案
grouped.to_csv('big_table_v3.csv', index=False)

# 顯示處理後的 DataFrame
grouped.head(20)

,Gene_ID,Transcript_Name,Type
0,WBGene00007063,2L52.1a.1,coding_transcript
1,WBGene00007063,2L52.1b.1,coding_transcript
2,WBGene00007064,2RSSE.1a.1,coding_transcript
3,WBGene00007064,2RSSE.1b.1,coding_transcript
4,WBGene00007064,2RSSE.1c.1,coding_transcript
5,WBGene00007065,3R5.1a.1,coding_transcript
6,WBGene00007065,3R5.1b.1,coding_transcript
7,WBGene00003525,4R79.1a.1,coding_transcript
8,WBGene00003525,4R79.1b.1,coding_transcript
9,WBGene00007067,4R79.2a.1,coding_transcript
